In [1]:
%matplotlib inline
import sys
sys.path.append('../../')
import pandas as pd
import numpy as np
import sqlite3 as lite
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from collections import OrderedDict
import time
#from modules.backtesting import BackTester
from modules.db_manager import open_file, product_info


In [2]:
raw_file =  open_file('h5py', 'training set/40day_trend_labeled_futures.h5', mode='r')

In [11]:
class Instrument:
    """
    상품별로 전략을 테스트 하는 클래스
    """

    def __init__(self, feed, pinfo, principal):
        self.feed = feed #pandas dataframe 상품 가격 데이터 
        self.pinfo = pinfo #xingapi로 받아온 상품 기초정보
        self.principal = principal
        self._statement = [] #총평가기록
        self._tradelog = [] #매매기록
        self._equitylog = [] #자산곡선

    def get_profit(self, position, entryprice, exitprice, lot=1, ptype='money'):
        """
        틱: (청산가격 - 진입가격)/틱단위
        손익계산: 랏수 * 틱가치 * 틱      
        """
        tick = position * (exitprice - entryprice)/self.pinfo['tick_unit']
        
        if ptype=='tick':
            return tick
        else:
            return lot * self.pinfo['tick_value']* tick
    
    def get_lot(self, unit):
        """
        랏계산: 유닛 / 개시증거금
        """
        lot = int(unit/self.pinfo['open_margin'])
        return lot if lot > 0 else 0

    
    def plot(self):
        tradelog = self.tradelog
        equitylog = self.equitylog
        fig, (ax) = plt.subplots(2,1, figsize=(12,10))
        ax[0].plot(self.feed.date, self.feed.close)
        for _,position, entrydate, exitdate in tradelog[['position','entrydate','exitdate']].itertuples():
            color = 'red' if position == 1 else 'green'
            ax[0].axvspan(entrydate, exitdate, facecolor=color, alpha=0.3)
        ax[1].plot(equitylog.date, equitylog.equity)
        ax[1].plot(equitylog.date, equitylog.equity.cummax())
        
        return ax[0]
    
    
    def report(self):
        equitylog = self.equitylog
        tradelog = self.tradelog
        equity = equitylog.equity
        clo_bal = equity.iloc[-1]
        timedelta = (equitylog.date.max() - equitylog.date.min()).days/365.25
        
        
        drawdown = (equity.cummax() - equity)/equity.cummax()
        mdd = drawdown.max()
        icagr = np.log(clo_bal/self.principal) /timedelta
        bliss = icagr/mdd
        cum_profit = ((equity.iloc[-1] / self.principal) -1 )
        win_rate = tradelog.profit[tradelog.profit >= 0].count()\
                    / tradelog.profit.count()
        max_profit = tradelog.profit.max()
        max_loss = tradelog.profit.min()
        profit_factor = abs(tradelog.profit[tradelog.profit >=0].sum()/\
                       tradelog.profit[tradelog.profit < 0].sum())
        num_trade = tradelog.entryid.max() / timedelta
        

        data = [[self.principal, clo_bal, cum_profit, bliss, icagr, mdd, 
                 profit_factor, win_rate, max_profit, max_loss, num_trade]]
        columns = ['투자금','최종자산','총손익','Bliss','ICAGR','MDD','손익비','승률',
                   '최대수익','최대손실','매매횟수(년)']
        report = pd.DataFrame(data, index=[self.pinfo['name']], columns=columns)
        report = report.style.format({
            '투자금': "{:,.1f}",
            '최종자산': "{:,.1f}",
            '총손익': "{:.2%}",
            'Bliss': "{:.4f}",
            'ICAGR': "{:.2%}",
            'MDD': "{:.2%}",
            '손익비': "{:.2f}",
            '승률': "{:.2%}",
            '최대수익': "{:,.2f}",
            '최대손실': "{:,.2f}",
            '연평균 매매횟수': "{:.2f}"
        })
        return report
    
    def run_trade(self):
        """
        feed의 신호로부터 실제 매매를 시뮬레이션 한다
        feed는 다음의 column을 반드시 포함한다.
        1. close: 당일 종가
        2. signal: 매매신호, 1 -> 진입 -1 -> 청산 0 -> none
        3. position: 포지션, 1 -> long -1 -> short 0 -> none
        4. price: 매매가격
        """
        entryid = 0
        open_lot = 0 #잔여 랏수
        clo_bal = self.principal #확정자산
        
        for date, close, signal, pos, price in self.feed[['close','signal','position','price']].itertuples():
            profit = 0
            
            #진입신호 발생
            if signal == 1:
                entryid += 1
                position = pos #진입 포지션
                #entrylot = self.get_lot(equity/20) #진입 랏수
                entrylot = 1
                entryprice = price #진입 가격
                entrydate = date #진입 날짜
                open_lot += entrylot #잔여 랏수
            
            #청산신호 발생
            if signal == -1:
                exitlot = entrylot #청산 랏수
                exitprice = price #청산 가격
                exitdate = date #청산 날짜
                open_lot -= exitlot #잔여 랏수
                
                tick_profit = self.get_profit(position, entryprice, exitprice, ptype='tick')
                profit = self.get_profit(position, entryprice, exitprice, exitlot)
                self._tradelog.append([entryid, position, entrylot, entrydate, entryprice, 
                                 exitlot, exitdate, exitprice, profit, tick_profit])
                                            
            
            #랏이 남아있으면 평가손익 계산 없으면 0
            open_profit = self.get_profit(position, entryprice, close, open_lot) if open_lot else 0
                
            clo_bal = clo_bal + profit
            equity = clo_bal + open_profit
            self._equitylog.append([date, clo_bal, open_profit, equity])        
    
    @property
    def equitylog(self):
        """
        평가 손익을 포함한 자산 테이블
        """
        columns = ['date','close balance','open profit','equity']
        equitylog = pd.DataFrame(self._equitylog, columns=columns)
        return equitylog[equitylog.date <= self.tradelog.exitdate.max()]
    
    @property
    def tradelog(self):
        """
        매매기록 테이블
        """
        columns = ['entryid','position','entrylot', 'entrydate', 'entryprice',
                   'exitlot', 'exitdate', 'exitprice', 'profit', 'tickprofit']
        log = pd.DataFrame(self._tradelog, columns=columns)
        #log['profit'] = log.position * log.exitlot * (log.exitprice - log.entryprice)/self.pinfo['tick_unit']
        log.insert(0, 'symbol', self.pinfo['group']) 
        return log

In [12]:
class BackTester:

    def __init__(self, feed, strategy=None, name='STRATEGY', principal=5000):
        
        self.feed = feed
        #self.columns = feed.attrs['columns'].split(';')
        self.name = name    
        
        self.products = product_info()#self._get_meta(BackTester.INFO_PATH)

        self.principal = principal
        self.insts = []

        if strategy:
            self.strategy = strategy           
        else: 
            self.strategy = self.default_strategy

        
    @property
    def statement(self):
        return self._statement

    
    def report(self, level=0):
        if level == 0:
            report = BackTester.get_result(self._statement, self.name)
            
        elif level == 1:
            report = BackTester.get_result(self._statement, self.name)
            report = report.append(pd.concat([item.report() for item in self.trades]))
        
        return report
    
    
    def summary(self, level=0):
        self.plot()
        return self.report(level)
    
    def run(self):
        print("trading started. it takes few minutes...")
        columns = self.feed.attrs['columns'].split(';')
        for idx, raw in enumerate(self.feed.values()):
            symbol = raw.attrs['symbol']
            #이베스트 미거래종목 제외
            if symbol == 'None' or not symbol:
                continue

            else:
                data = pd.DataFrame(raw.value[:,1:], index=raw.value[:,0].astype('M8[s]'), columns=columns[1:])
                data.insert(0,'date', data.index)
                
                inst = Instrument(data, self.products[symbol], self.principal)
                self.strategy(inst)
                inst.run_trade()
                self.insts.append(inst)

                print(f'\rprocessing..({idx})', end='', flush=True)
        
        #self._statement = pd.concat([trade.statement for trade in self.trades])[['name','entrydate','exitdate','profit']]
        #self._statement.sort_values('entrydate', inplace=True)
        #self._statement.reset_index(drop=True, inplace=True)
        #self._statement['cumprofit'] = self._statement.profit.cumsum()
        #self._statement['drawdown'] = self._statement.cumprofit.cummax() - self._statement.cumprofit
        
        print("\nDone")

    
    def plot(self):
        x = self._statement.entrydate.values
        y1 = self._statement.cumprofit.values
        y2 = self._statement.cumprofit.cummax().values
        fig, ax = plt.subplots(1,1, figsize=(10, 8))
        ax.fill_between(x, 0, y1, where=y1>=0, facecolor='green',alpha=0.4, interpolate=True)
        ax.fill_between(x, 0, y1, where=y1<0, facecolor='red', alpha=0.8, interpolate=True)
        ax.fill_between(x,y1, y2, color='grey', alpha=0.2)

        #labels
        ax.set_title('Cumulative Profit')
        ax.set_xlabel('Date')
        ax.set_ylabel('Profit')
        ax.yaxis.set_label_position("right")
        
        #style
        ax.grid(linestyle='--')
        ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
        ax.yaxis.tick_right()
        fig.autofmt_xdate()
        
        plt.show()    
    
    @staticmethod
    def get_result(statement, name):
        #receipt = self.receipt
        #mdd = 100*(statement.drawdown/statement.cumprofit.cummax()).max()
        mdd = statement.drawdown.max()
        cum_profit = statement.cumprofit.iloc[-1]
        ave_profit = statement.profit.mean()
        rng = statement.exitdate.max() - statement.entrydate.min()
        rng = rng.days/365.25
        cagr = pow(cum_profit, 1/rng) - 1 if cum_profit > 0 else 0
        win_rate = 100 * statement.profit[statement.profit >= 0].count() \
                    / statement.profit.count()
        max_loss = statement.profit.min()
        max_profit = statement.profit.max()
        #ave_win = statement.profit[receipt.profit >= 0].mean()
        #ave_lose = statement.profit[receipt.profit < 0].mean()
        profit_factor = abs(statement.profit[statement.profit >=0].sum()/\
                       statement.profit[statement.profit < 0].sum())
        num_trade = len(statement) / rng # 연평균 매매 횟수

        data = [[cum_profit, cagr, profit_factor, mdd, win_rate, max_profit, max_loss, num_trade]]
        columns = ['총손익(틱)','CAGR(%)','손익비','MDD(틱)','승률(%)', '최대수익(틱)','최대손실(틱)','연평균 매매횟수']
        return pd.DataFrame(data, index=[self.pinfo['name']], columns=columns)
    
    @staticmethod
    def default_strategy(trade):
        """
        *규칙*
        - feed data는 다음의 column들을 반드시 포함해야 한다.
        1. signal(매매 신호); 1: 진입, -1: 청산, 0: none
        2. position: 1: long, -1: short, 0: none
        3. price: 진입 또는 청산 가격
        
        *예시* buy only MA cross system
        진입: 20종가이평이 60종가이평 돌파상승 한 다음날 시가 진입
        청산: 진입한 투자금(증거금) 대비 10% 이상 평가손실시 청산
        """
        data = trade.feed
        data['ma20'] = data.close.rolling(30).mean()
        data['ma60'] = data.close.rolling(200).mean()
        data['cross'] = (data.ma20 > data.ma60).astype('int')
        data['signal'] = data.cross.diff().shift(1)
        #data['signal'] = np.where(data['signal'] == 1, 1, 0)
        data['position'] = np.where(data['signal'] == 1, 1, 0) 
        data['price'] = np.where((data['signal']==1) | (data['signal']==-1), data['open'], 0)
        data.dropna(inplace=True)
        """
        entryid = 0
        open_lot = 0 #잔여 랏수
        clo_bal = trade.principal #확정자산
        
        for date, open, close, signal in data[['open','close','signal']].itertuples():
            profit = 0
            
            if signal == 1:
                entryid += 1
                position = LONG #진입 포지션
                entrylot = trade.get_lot(equity/20) #진입 랏수
                #entrylot = 1
                entryprice = open #진입 가격
                entrydate = date #진입 날짜
                open_lot += entrylot #잔여 랏수
                
            if signal == -1:
                exitlot = entrylot #청산 랏수
                exitprice = open #청산 가격
                exitdate = date #청산 날짜
                open_lot -= exitlot #잔여 랏수
                
                tick_profit = trade.get_profit(position, entryprice, exitprice, ptype='tick')
                profit = trade.get_profit(position, entryprice, exitprice, exitlot)
                trade._tradelog.append([entryid, position, entrylot, entrydate, entryprice, 
                                 exitlot, exitdate, exitprice, profit, tick_profit])
                                            
            
            #랏이 남아있으면 평가손익 계산 없으면 0
            open_profit = trade.get_profit(position, entryprice, close, open_lot) if open_lot else 0
                
            clo_bal = clo_bal + profit
            equity = clo_bal + open_profit
            trade._equitylog.append([date, clo_bal, open_profit, equity])
        """

In [13]:
#columns = raw_file.attrs['columns'].split(';')
back = BackTester(feed=raw_file, principal=100000)

In [14]:
ret = back.run()

trading started. it takes few minutes...
processing..(61)
Done


In [82]:
a = back.insts[0]

In [83]:
a.report()

,투자금,최종자산,총손익,Bliss,ICAGR,MDD,손익비,승률,최대수익,최대손실,매매횟수(년)
Australian Dollar,"100,000.0","121,890.0",21.89%,0.1273,2.24%,17.62%,2.59,42.86%,"13,170.00","-6,780.00",0.793529


In [110]:
a, b = back.insts[0].equitylog, back.insts[1].equitylog

In [111]:
a.set_index('date',inplace=True)

In [114]:
b.set_index('date',inplace=True)

In [118]:
s=pd.concat([a, b], axis=1)

In [127]:
s['open profit']

,open profit,open profit
date,,
2005-12-07,0.0,NaN
2005-12-08,0.0,NaN
2005-12-09,0.0,NaN
2005-12-12,0.0,NaN
2005-12-13,0.0,0.0
2005-12-14,0.0,0.0
2005-12-15,0.0,0.0
2005-12-16,0.0,0.0
2005-12-19,0.0,0.0


In [100]:
grp = s.groupby('date')['open profit'].sum()

In [105]:
grp.max()

42975.999999999993

In [107]:
back.insts[0].equitylog['open profit'].max()

24480.0

In [67]:
back.insts[0].plot()

TypeError: invalid type comparison

In [55]:
b = B()

In [56]:
b.func(a.var)

In [57]:
a.var

[90]